In [1]:
import collections
import numpy as np

start_token = 'B'
end_token = 'E'

def process_poems(file_name):
    poems=[]
    with open(file_name, 'r', encoding='utf-8', ) as f:
        for line in f.readlines():
            try:
                title, content = line.strip().split(':')
                content = content.replace(' ','')
                if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content or \
                        start_token in content or end_token in content:
                    continue
                if len(content) < 5 or len(content) > 79:
                    continue
                content = start_token + content + end_token
                poems.append(content)
            except ValueError as e:
                pass
    all_words = [word for poem in poems for word in poem]
    print(all_words)
    counter = collections.Counter(all_words)
    words = sorted(counter.keys(), key=lambda x: counter[x], reverse=True)
    
    words.append(' ')
    L = len(words)
    word_int_map = dict(zip(words, range(L)))
    poems_vector = [list(map(lambda word: word_int_map.get(word, L), poem)) for poem in poems]
    
    return poems_vector, word_int_map, words

def generate_batch(batch_size, poems_vec, word_to_int):
    n_chunk = len(poems_vec) // batch_size
    x_batches = []
    y_batches = []
    for i in range(n_chunk):
        start_index = i*batch_size
        end_index = start_index+batch_size
        
        batches = poems_vec[start_index: end_index]
        length = max(map(len, batches))
        x_data = np.full((batch_size, length), word_to_int[' '], np.int32)
        for row,batch in enumerate(batches):
            x_data[row, :len(batch)] = batch
        y_data = np.copy(x_data)
        y_data[:, :-1] = x_data[:, 1:]
        """
        x_data             y_data
        [6,2,4,6,9]       [2,4,6,9,9]
        [1,4,2,8,5]       [4,2,8,5,5]
        """
        x_batches.append(x_data)
        y_batches.append(y_data)
        
    return x_batches, y_batches

import tensorflow as tf
import numpy as np

def rnn_model(model, input_data, output_data, vocab_size, rnn_size=128, num_layers = 2, 
              batch_size=64, learning_rate=0.01):
    """
    construct rnn seq2seq model.
    :param model: model class
    :param input_data: input data placeholder
    :param output_data: output data placeholder
    :param vocab_size:
    :param rnn_size:
    :param num_layers:
    :param batch_size:
    :param learning_rate:
    :return:
    """
    end_points = {}
    if model == 'rnn':
        cell_fun = tf.contrib.rnn.BasicRNNCell
    elif model=='gru':
        cell_fun = tf.contrib.rnn.GRUCell
    elif model == 'lstm':
        cell_fun = tf.contrib.rnn.BasicLSTMCell
    cell =  cell_fun(rnn_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    
    if output_data is not None:
        initial_state = cell.zero_state(batch_size, tf.float32)
    else:
        initial_state = cell.zero_state(1, tf.float32)
    
    with tf.device("/cpu:0"):
        embedding = tf.get_variable('embedding', initializer=tf.random_uniform([vocab_size+1, rnn_size],
                                                                               -1.0, -1.0))
        inputs = tf.nn.embedding_lookup(embedding, input_data)
    # [batch_size, ?, rnn_size] = [64, ?, 128]
    outputs, last_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
    output = tf.reshape(outputs,[-1, rnn_size])
    
    weithts = tf.Variable(tf.truncated_normal([rnn_size, vocab_size+1]))
    bias = tf.Variable(tf.zeros(shape=[vocab_size+1]))
    logits = tf.nn.bias_add(tf.matmul(output, weithts), bias=bias)
    # [?, vocab_size+1]
    
    if output_data is not None:
        labels = tf.one_hot(tf.reshape(output_data,[-1]), depth=vocab_size+1)
        
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        total_loss = tf.reduce_mean(loss)
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(total_loss)
        
        end_points['initial_state'] = initial_state
        end_points['output'] = output
        end_points['train_op'] = train_op
        end_points['total_loss'] = total_loss
        end_points['loss'] = loss
        end_points['last_state'] = last_state
    else:
        prediction = tf.nn.softmax(logits)
        end_points['initial_state'] = initial_state
        end_points['last_state'] = last_state
        end_points['prediction'] = prediction
    return end_points

import os
import tensorflow as tf

tf.app.flags.DEFINE_integer('batch_size',64, 'batch_size.')
tf.app.flags.DEFINE_float('learning_rate',0.01, 'learning_rate.')
tf.app.flags.DEFINE_string('model_dir', os.path.abspath('./model2'), 'model save path.')
tf.app.flags.DEFINE_string('file_path', os.path.abspath('./data/poems.txt'), 'file name of poems.')
tf.app.flags.DEFINE_string('model_prefix', 'poems', 'model save prefix.')
tf.app.flags.DEFINE_integer('epoches', 50, 'train how many epochs.')
tf.app.flags.DEFINE_string('f', '', 'kernel')
    
FLAGS = tf.app.flags.FLAGS

def run_training():
    if not os.path.exists(FLAGS.model_dir):
        os.makedirs(FLAGS.model_dir)
    poem_vector, word_to_int, vocabularies = process_poems(FLAGS.file_path)
    batches_inputs, batches_outputs = generate_batch(FLAGS.batch_size, poem_vector, word_to_int)
    
    input_data = tf.placeholder(tf.int32, [FLAGS.batch_size, None])
    output_targets = tf.placeholder(tf.int32, [FLAGS.batch_size, None])
    
    end_points = rnn_model(model='lstm', input_data=input_data, output_data=output_targets,
                           vocab_size=len(vocabularies), rnn_size=128, num_layers=2, 
                           batch_size=64,learning_rate=FLAGS.learning_rate)
    saver = tf.train.Saver(tf.global_variables())
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    
    with tf.Session() as sess:
        sess.run(init_op)
        start_epoch = 0
        checkpoint = tf.train.latest_checkpoint(FLAGS.model_dir)
        if checkpoint:
            saver.restore(sess, checkpoint)
            print("## restore from the checkpoint {0}".format(checkpoint))
            start_epoch += int(checkpoint.split('-')[-1])
        print("## start training ...")
        try:
            n_chunk = len(poem_vector) // FLAGS.batch_size
            print("n_chunk: %d" % n_chunk)
            for epoch in range(start_epoch, FLAGS.epoches):
                n = 0
                for batch in range(n_chunk):
                    loss, _, _ = sess.run([end_points['total_loss'],
                                        end_points['last_state'],
                                        end_points['train_op']
                                       ],feed_dict={input_data:batches_inputs[n],output_targets:batches_outputs[n]})
                    
                    n +=1
                    if batch % 50 ==0:
                        print("Epoch: %d , batch: %d, training loss: %.6f" % (epoch, batch, loss))
                if epoch % 6 ==0:
                    saver.save(sess, os.path.join(FLAGS.model_dir, FLAGS.model_prefix), global_step=epoch)
        except KeyboardInterrupt:
            print('## Interrupt manually, try saving checkpoint for now...')
            saver.save(sess, os.path.join(FLAGS.model_dir, FLAGS.model_prefix), global_step=epoch)
            print('## Last epoch were saved, next time will start from epoch {}.'.format(epoch))

def main(_):
    #print("hello")
    run_training()

#tf.app.run()

In [2]:
import tensorflow as tf
import numpy as np

def to_word(predict, vocabs):
    predict = predict[0]
    predict /= np.sum(predict)
    sample = np.random.choice(np.arange(len(predict)), p = predict)
    if sample > len(vocabs):
        return vocabs[-1]
    else:
        return vocabs[sample]

def gen_poem(begin_word):
    batch_size = 1
    print("## loading corpus from %s " % FLAGS.model_dir)
    poems_vector, word_int_map, vocabularies = process_poems(FLAGS.file_path)
    input_data = tf.placeholder(tf.int32, [batch_size, None])
    end_points = rnn_model(model='lstm', input_data=input_data, output_data=None, 
                          vocab_size=len(vocabularies),rnn_size=128, num_layers=2,
                          batch_size=64,learning_rate=FLAGS.learning_rate)
    saver = tf.train.Saver(tf.global_variables())
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    with tf.Session() as sess:
        sess.run(init_op)
        check_point = tf.train.latest_checkpoint(FLAGS.model_dir)
        saver.restore(sess, check_point)
        x = np.array([list(map(word_int_map.get, start_token))])
        #print(x)
        #x = x = np.array([[word_int_map[begin_word]]])
        [predict, last_state] = sess.run([end_points['prediction'], end_points['last_state']], 
                                         feed_dict={input_data:x})
        word = begin_word or to_word(predict, vocabularies)
        poem_ = ''
        i = 0
        while word != end_token:
            poem_ +=word
            i +=1
            #print(i)
            if i>124:
                break
            if word == ' ':
                x = np.array([list(map(word_int_map.get, start_token))])
            else:
                x = np.array([[word_int_map[word]]])
            [predict, last_state] = sess.run([end_points['prediction'], end_points['last_state']],
                                             feed_dict={input_data: x, 
                                                        end_points['initial_state']: last_state})
            word = to_word(predict, vocabularies)
            
            
        return poem_

def pretty_print_poem(poem_):
    poem_sentences = poem_.split("。")
    for s in poem_sentences:
        if s != '':
            print(s+"。")



begin_char = input('## please input the first character:')
poem = gen_poem(begin_char)
#print(poem)
pretty_print_poem(poem_=poem)

## please input the first character:雨
## loading corpus from /media/liangsh/新加卷1/天池阿里云/天池课程练习tf/循环神经网络/tensorflow_poems/model2 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
INFO:tensorflow:Restoring parameters from /media/liangsh/新加卷1/天池阿里云/天池课程练习tf/循环神经网络/tensorflow_poems/model2/poems-6


InternalError: Blas GEMM launch failed : a.shape=(1, 256), b.shape=(256, 512), m=1, n=512, k=256
	 [[node rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul (defined at <ipython-input-1-659c66c16519>:96)  = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/concat, rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul/Enter)]]
	 [[{{node rnn/while/Exit_6/_33}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_199_rnn/while/Exit_6", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/pytho